In [ ]:
import sys
import os
import random
import datetime
import re
import math
import logging
from collections import OrderedDict
import multiprocessing
import numpy as np
import tensorflow as tf
import  keras
import keras.backend as K
import keras.layers as KL
import keras.engine as KE
import keras.models as KM

import index_tracker as track   
import csv
from keras.callbacks import LearningRateScheduler
import cv2
import scipy
import h5py
import skimage
import time

# set seed
from numpy.random import seed
seed(1234)
from tensorflow import set_random_seed
set_random_seed(1234)

from siammask import utils, rpn as rp, data_generator as dg, visualize, metrics
from siammask.config import Config


from siammask import siammask_model as modellib
import dicom_to_numpy as dtn
import matplotlib.pyplot as plt

from siammask import testing_code

**1. Set training hyperparameters**

In [ ]:
class TumorConfig(Config):    
    NAME = 'lung_tumor'    
    LOSS_WEIGHTS = {
        "rpn_class_loss": 1.,
        "rpn_bbox_loss": 1.,   
        "rpn_centerness_loss": 1.,        
        "rpn_mask_loss": 1.        
    }
    LEARNING_RATE = 0.001
    LEARNING_MOMENTUM = 0.9
    IMAGES_PER_GPU = 16
    TRAIN_BN = None
                 
    IMAGE_HEIGHT = 384
    IMAGE_WIDTH = 512
    IMAGE_SHAPE = [384, 512, 1]
    STEPS_PER_EPOCH = 1000
    VALIDATION_STEPS = 181
    
    BACKBONE_STRIDES = [8]
    RPN_ANCHOR_SCALES = [64]
    
    POST_NMS_ROIS_TRAINING = 1024
    POST_NMS_ROIS_INFERENCE = 512
    RPN_TRAIN_ANCHORS_PER_IMAGE = 256
    RPN_NMS_THRESHOLD = 0.8
    BBOX_LOSS_LAMDA_D = 10
    BBOX_LOSS_LAMDA_C = 1
    
    TRAIN_MASKS_PER_IMAGE = 20
    
    USE_BN = True
    USE_DP = False
    
    ANCHORLESS = True 
    SIAMRPN = True 
    CROP = True
    
    MASK = False
    REFINE = False    
    STAGE4 = True
    SHARED = False
    BACKBONE = "resnet50"
    
    BBOX_RATIO = 0.7
    
    BBOX_SIGMA = 1.0
    
    KERNEL_IMAGE_SIZE = 128
    IMAGE_CHANNEL_COUNT = 3
    if CROP:
        FEATURES_SHAPE = [25, 25]
    else:
        FEATURES_SHAPE = [17, 17]   


# SET UP CONFIG
config = TumorConfig()

**2. Load training and validation datasets**

In [ ]:
data_path = ''
patient_name = 'patient2'
model = 'siammask'
data_mode = 'decorrelated'
drr_data_path = data_path + patient_name + '/models/'+ model + '/' + data_mode +'/dataset/'
# Directory to save logs and trained model
LOG_DIR =  data_path + patient_name + '/models/' + model + '/' + data_mode +'/logs/'


gans=''
imgs = dtn.load_data(drr_data_path + 'train_imgs'+gans+'.hdf5', 'train_imgs'+gans)
labs = dtn.load_data(drr_data_path + 'train_labs.hdf5', 'train_labs')
test_imgs = dtn.load_data(drr_data_path + 'test_imgs.hdf5', 'test_imgs')
test_labs = dtn.load_data(drr_data_path + 'test_labs.hdf5', 'test_labs')

**3. Preprocess loaded datasets**

In [ ]:
for i in range(labs.shape[0]):
    for j in range(labs.shape[1]):
        labs[i, j] = fit_ellipse(labs[i, j])
for i in range(test_labs.shape[0]):
    for j in range(test_labs.shape[1]):
        test_labs[i, j] = fit_ellipse(test_labs[i, j])
        
imgs = (imgs+1)/2.
test_imgs = (test_imgs+1)/2.

**4. Define training model**

In [ ]:
model = modellib.SiamMask(mode="training", config=config,
                                      model_dir=LOG_DIR)

**5. Train model**

In [ ]:
def lr_scheduler(epoch, lr):
    if epoch > 0 and epoch<=10 and lr < 1e-2:
        lr = lr + lr/epoch
    elif epoch > 10 and lr > 1e-5:
        k = 0.05
        lr = lr*np.exp(-k*epoch)
    return lr

model.train(imgs, labs, 
            test_imgs, test_labs,
            learning_rate=config.LEARNING_RATE,
            epochs=30,
            layers='all',
            custom_callbacks = [lrs])